# Finite Difference Method (FDM)

이번 챕터는 파생상품의 프라이싱과 헤징에 사용되는 유한차분법(Finite Difference Method, FDM)에 대해 다룹니다.

유한차분법은 모든 파생상품의 가격 $V(S,T)$ 가 따르는 미분방정식인 블랙 숄즈 방정식에서 시작합니다.

블랙 숄즈 방정식은 다음과 같습니다.

$${\partial V\over\partial T} + (r-q)S {\partial V\over\partial S} + {1 \over\ 2} \sigma^2 S^2 {\partial^2 V\over\partial S^2} = rV$$

이때, $T$는 잔여만기, $S$는 기초자산의 가격, $\sigma$는 기초자산의 변동성, $r$과 $q$는 각각 연속 무위험 수익률과 연속 배당률을 의미합니다.

모든 파생상품의 가격은 위 방정식의 만기 조건에 해당 파생상품의 만기 payoff를 대입하고 위 방정식을 푸는 것으로 계산이 가능합니다.

하지만 복잡한 payoff를 가지는 파생상품은 위 방정식을 이용하여 가격을 구하는 것이 불가능에 가깝습니다.

따라서 본 챕터에서는 위의 미분방정식을 연립방정식의 형태로 치환하여 파생상품의 가격을 구하는 FDM을 구현합니다.

FDM에서는 미분 $f'(x)$가 차분 ${f(x+h)-f(x) \over\ h}$의 극한임을 이용하여, 블랙 숄즈 방정식의 미분으로 나타내어진 항을 모두 차분의 형태로 치환합니다.

그 과정에서 미분으로 나타내어진 항은 다음과 같이 치환됩니다.

$${\partial V\over\partial T} \approx {V(S,T+dt)-V(S,T) \over\ dt}$$

$${\partial V\over\partial S} \approx {V(S+dS,T)-V(S-dS,T) \over\ 2dS}$$

$${\partial^2 V\over\partial S^2} \approx {V(S+dS,T)-2V(S,T)+V(S-dS,T) \over\ dS^2}$$

블랙 숄즈 방정식에 위의 치환을 적용하면 다음과 같이 표현됩니다.

$$ a_1(S) V(S-ds,T) + a_2(S) V(S,T) + a_3(S) V(S+ds,T) = {V(S,T+dt) \over\ dt}, 0<S<S_{max}$$

이때, 모든 $T$에 대해 경계 조건, 즉 $V(0,T), V(S,T)$는 다음 조건을 만족하게 설정됩니다.
$$V''(0,T)= V''(S,T) = 0$$

행렬을 이용하여 위 관계식을 표현하면 다음과 같습니다.

$$ \begin{bmatrix}
2a_1(dS)+a_2(dS)&-a_1(dS)+a_3(dS) & 0 & \cdots & 0 \\
a_1(2dS) &a_2(2dS) & a_3(2dS) & \cdots & \vdots \\
0&a_1(3dS)&a_2(3dS)& \ddots & \vdots \\ 
\vdots&\vdots&\vdots& \vdots & \vdots \\ 
0 & \cdots & 0 & a_1(S_{max}-dS)-a_3(S_{max}-dS) & a_2(S_{max}-dS)+2a_3(S_{max}-dS)
\end{bmatrix} 
\begin{bmatrix} V(ds,T) \\ V(2ds,T) \\ v \dots \\ V(S_{max}-dS,T)\end{bmatrix}
 
= \begin{bmatrix}V(ds,T+dt) \\ V(2ds,T+dt) \\ \vdots\\ V(S_{max}-dS,T+dt)
\end{bmatrix}$$ 

위 식을 단순화하면 $AS_{T} = S_{T+dt}$ 가 되며, $S_{T} = A^{-1} S_{T+dt}$ 가 성립됩니다.

이제 만기 시점에서의 파생상품의 payoff, 즉 $V(S,T_{max})$를 대입한 후, 위 선형방정식을 풀어 $T$를 $dt$씩 줄여가며, 각 시점에서의 파생상품 가치를 계산하여 최종적으로 초기에서의 파생상품의 가치를 계산합니다.

이때, $A$는 삼중 대각행렬이므로, [Thomas' Theorem](https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm) 을 사용하여 빠르게 위 선형방정식을 풀 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns

